In [1]:
using Pkg

In [2]:
Pkg.add("ChainRulesCore")

    Updating registry at `C:\Users\amans\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [3]:
Pkg.add("ChainRules")

   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [4]:
Pkg.add("ChainRulesTestUtils")

   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [5]:
using ChainRulesCore
using ChainRules
using ChainRulesTestUtils

In [6]:
using Zygote

In [7]:
y, back = Zygote.pullback(sin, 0.5);

In [8]:
y

0.479425538604203

In [9]:
sin(0.5)

0.479425538604203

In [10]:
dsin(x) = (sin(x), ȳ -> (ȳ * cos(x),))

dsin (generic function with 1 method)

In [11]:
dsin(0.5)

(0.479425538604203, var"#1#2"{Float64}(0.5))

In [12]:
# Additionally, for signalling semantics, we distinguish between two tangent types representing a zero tangent. 
# NoTangent type represent situtations in which the tangent space does not exist, e.g. an index into an array can not be perturbed. 
# ZeroTangent is used for cases where the tangent happens to be zero, e.g. because the primal argument is not used in the computation.

In [13]:
using ChainRulesCore

function foo(x)
    a = sin(x)
    b = 0.2 + a
    c = asin(b)
    return c
end

# Define rules (alternatively get them for free via `using ChainRules`)
@scalar_rule(sin(x), cos(x))
@scalar_rule(+(x, y), (1.0, 1.0))
@scalar_rule(asin(x), inv(sqrt(1 - x^2)))

In [14]:
#### Find dfoo/dx via rrules
#### First the forward pass, gathering up the pullbacks
x = 3;
a, a_pullback = rrule(sin, x);
b, b_pullback = rrule(+, 0.2, a);
c, c_pullback = rrule(asin, b)

#### Then the backward pass calculating gradients
c̄ = 1;                    # ∂c/∂c
_, b̄ = c_pullback(c̄);     # ∂c/∂b = ∂c/∂b ⋅ ∂c/∂c
_, _, ā = b_pullback(b̄);  # ∂c/∂a = ∂c/∂b ⋅ ∂b/∂a
_, x̄ = a_pullback(ā);     # ∂c/∂x = ∂c/∂a ⋅ ∂a/∂x
x̄

-1.0531613736418153

In [15]:
#### Find dfoo/dx via frules
x = 3;
ẋ = 1;              # ∂x/∂x
nofields = ZeroTangent();  # ∂self/∂self

a, ȧ = frule((nofields, ẋ), sin, x);                    # ∂a/∂x = ∂a/∂x ⋅ ∂x/∂x 
b, ḃ = frule((nofields, ZeroTangent(), ȧ), +, 0.2, a);  # ∂b/∂x = ∂b/∂a ⋅ ∂a/∂x
c, ċ = frule((nofields, ḃ), asin, b);                   # ∂c/∂x = ∂c/∂b ⋅ ∂b/∂x
ċ 

-1.0531613736418153

In [16]:
Pkg.add("FiniteDifferences")

   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [17]:
Pkg.add("ForwardDiff")

   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [18]:
#### Find dfoo/dx via FiniteDifferences.jl
using FiniteDifferences
central_fdm(5, 1)(foo, x)
# output
# -1.0531613736418257

#### Find dfoo/dx via ForwardDiff.jl
using ForwardDiff
ForwardDiff.derivative(foo, x)
# output
# -1.0531613736418153

#### Find dfoo/dx via Zygote.jl
using Zygote
Zygote.gradient(foo, x)

(-1.0531613736418153,)

In [19]:
Pkg.add("ImageTransformations")

   Resolving package versions...
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\amans\.julia\environments\v1.7\Manifest.toml`


In [28]:
import Pkg

In [29]:
Pkg.develop("DiffImages")

LoadError: The following package names could not be resolved:
 * DiffImages (not found in project, manifest or registry)


In [20]:
using DiffImages, ImageTransformations, CoordinateTransformations, ImageCore, FileIO, StaticArrays

LoadError: ArgumentError: Package DiffImages not found in current path:
- Run `import Pkg; Pkg.add("DiffImages")` to install the DiffImages package.
